In [36]:
import pandas as pd 
import numpy as np
import sqlite3

conn = sqlite3.connect('example.db')
# df = pd.read_csv('voters.csv')
# df.to_sql('voters', conn, if_exists='replace', index=False)
def rsq(query: str) -> pd.DataFrame:
    return pd.read_sql_query(query, conn)

In [37]:
rsq("""SELECT
gender,
-- Count the number of voters for each group
COUNT(*) AS voters,
-- Calculate the total number of votes per group
SUM(total_votes) AS total_votes
FROM voters
GROUP BY gender;""")

,gender,voters,total_votes
0,F,91,9193
1,M,105,10708


In [38]:
rsq("""SELECT 
	company,
	-- Calculate the average cocoa percent
	AVG(cocoa_percent) AS avg_cocoa,
	-- Calculate the minimum rating received by each company
	MIN(rating) AS min_rating,
	-- Calculate the maximum rating received by each company
	MAX(rating) AS max_rating
FROM ratings
GROUP BY company
-- Order the values by the maximum rating
ORDER BY max_rating DESC;""")

,company,avg_cocoa,min_rating,max_rating
0,Bonnat,0.755556,2.75,5.0
1,Askinosie,0.720000,3.00,5.0
2,hello cocoa,0.655000,3.25,4.0
3,Zak's,0.683333,3.25,4.0
4,Woodblock,0.700000,1.75,4.0
...,...,...,...,...
408,Park 75,0.650000,2.00,2.0
409,Metropolitan,0.700000,2.00,2.0
410,Dole (Guittard),0.700000,2.00,2.0
411,Rain Republic,0.700000,1.00,1.0


In [ ]:
SELECT
first_name,
last_name,
total_votes AS votes,
-- Select the number of votes of the next voter
LEAD(total_votes) OVER (ORDER BY total_votes) AS votes_next_voter,
-- Calculate the difference between the number of votes
LEAD(total_votes) OVER (ORDER BY total_votes) - total_votes AS votes_diff
FROM voters
WHERE country = 'France'
ORDER BY total_votes;

In [40]:
rsq("""SELECT
first_name,
last_name,
total_votes AS votes,
-- Select the number of votes of the next voter
LEAD(total_votes) OVER (ORDER BY total_votes) AS votes_next_voter,
-- Calculate the difference between the number of votes
LEAD(total_votes) OVER (ORDER BY total_votes) - total_votes AS votes_diff
FROM voters
WHERE country = 'France'
ORDER BY total_votes;""")

,first_name,last_name,votes,votes_next_voter,votes_diff
0,Roger,Rai,49,56.0,7.0
1,Meredith,Vance,56,73.0,17.0
2,Max,Ruiz,73,90.0,17.0
3,Christine,Chander,90,105.0,15.0
4,Jose,Hughes,105,124.0,19.0
5,Ana,Price,124,140.0,16.0
6,Benjamin,Hughes,140,148.0,8.0
7,Andres,Nath,148,182.0,34.0
8,Logan,Anderson,182,NaN,NaN


In [ ]:
SELECT
broad_bean_origin AS bean_origin,
rating,
cocoa_percent,
-- Retrieve the cocoa % of the bar with the previous rating
LAG(cocoa_percent)
OVER(PARTITION BY broad_bean_origin ORDER BY rating ) AS percent_lower_rating
FROM ratings
WHERE company = 'Fruition'
ORDER BY broad_bean_origin, rating ASC;

In [39]:
rsq("""SELECT
broad_bean_origin AS bean_origin,
rating,
cocoa_percent,
-- Retrieve the cocoa % of the bar with the previous rating
LAG(cocoa_percent)
OVER(PARTITION BY broad_bean_origin ORDER BY rating ) AS percent_lower_rating
FROM ratings
WHERE company = 'Fruition'
ORDER BY broad_bean_origin, rating ASC;""")

,bean_origin,rating,cocoa_percent,percent_lower_rating
0,Bolivia,3.75,0.74,NaN
1,Costa Rica,1.50,0.66,NaN
2,Costa Rica,3.00,0.70,0.66
3,Costa Rica,3.00,0.74,0.70
4,Dominican Republic,3.25,0.70,NaN
5,Ecuador,3.25,0.75,NaN
6,Peru,2.50,0.76,NaN
7,Peru,3.25,0.70,0.76
8,Trinidad,2.00,0.70,NaN


In [ ]:
SELECT
first_name + ' ' + last_name AS name,
country,
birthdate,
-- Retrieve the birthdate of the oldest voter per country
FIRST_VALUE(birthdate)
OVER (PARTITION BY country ORDER BY birthdate) AS oldest_voter,
-- Retrieve the birthdate of the youngest voter per country
LAST_VALUE(birthdate)
OVER (PARTITION BY country ORDER BY birthdate ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
) AS youngest_voter
FROM voters
WHERE country IN ('Spain', 'USA');

In [44]:
rsq("""SELECT
(first_name || ' ' || last_name) AS name,
country,
birthdate,
-- Retrieve the birthdate of the oldest voter per country
FIRST_VALUE(birthdate)
OVER (PARTITION BY country ORDER BY birthdate) AS oldest_voter,
-- Retrieve the birthdate of the youngest voter per country
LAST_VALUE(birthdate)
OVER (PARTITION BY country ORDER BY birthdate ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
) AS youngest_voter
FROM voters
WHERE country IN ('Spain', 'USA');""")

,name,country,birthdate,oldest_voter,youngest_voter
0,Caroline Griffin,Spain,1981-03-20,1981-03-20,1988-03-21
1,Christopher Jackson,Spain,1981-04-15,1981-03-20,1988-03-21
2,Raul Raji,Spain,1981-04-25,1981-03-20,1988-03-21
3,Karen Cai,Spain,1981-05-03,1981-03-20,1988-03-21
4,Jose Scott,Spain,1981-05-13,1981-03-20,1988-03-21
5,Nancy Garcia,Spain,1982-02-13,1981-03-20,1988-03-21
6,Joshua Walker,Spain,1982-03-21,1981-03-20,1988-03-21
7,Hector Romero,Spain,1982-04-25,1981-03-20,1988-03-21
8,Tiffany Ye,Spain,1983-01-19,1981-03-20,1988-03-21
9,Jada Rivera,Spain,1983-02-13,1981-03-20,1988-03-21


In [ ]:
DECLARE @number1 DECIMAL(18,2) = -5.4;
DECLARE @number2 DECIMAL(18,2) = 7.89;
DECLARE @number3 DECIMAL(18,2) = 13.2;
DECLARE @number4 DECIMAL(18,2) = 0.003;

DECLARE @result DECIMAL(18,2) = @number1 * @number2 - @number3 - @number4;
SELECT
@result AS result,
-- Show the absolute value of the result
ABS(@result) AS abs_result,
-- Find the sign of the result
SIGN(@result) AS sign_result;



In [45]:
rsq("""-- Declare and assign variables
WITH vars AS (
  SELECT -5.4 AS number1, 7.89 AS number2, 13.2 AS number3, 0.003 AS number4
)
-- Calculate the result and show absolute value and sign
SELECT 
  (number1 * number2) - number3 - number4 AS result,
  ABS((number1 * number2) - number3 - number4) AS abs_result,
  SIGN((number1 * number2) - number3 - number4) AS sign_result
FROM vars;""")

,result,abs_result,sign_result
0,-55.809,55.809,-1


In [ ]:
SELECT
rating,
-- Round-up the rating to an integer value
CEILING(rating) AS round_up,
-- Round-down the rating to an integer value
FLOOR(rating) AS round_down,
-- Round the rating value to one decimal
ROUND(rating, 1) AS round_onedec,
-- Round the rating value to two decimals
ROUND(rating, 2) AS round_twodec
FROM ratings;

In [48]:
rsq("""SELECT
rating,
-- Round-up the rating to an integer value
CEILING(rating) AS round_up,
-- Round-down the rating to an integer value
cast(FLOOR(rating) as integer) AS round_down,
-- Round the rating value to one decimal
ROUND(rating, 1) AS round_onedec,
-- Round the rating value to two decimals
ROUND(rating, 2) AS round_twodec
FROM ratings;""")

,rating,round_up,round_down,round_onedec,round_twodec
0,2.75,3.0,2,2.8,2.75
1,2.75,3.0,2,2.8,2.75
2,2.75,3.0,2,2.8,2.75
3,2.75,3.0,2,2.8,2.75
4,3.00,3.0,3,3.0,3.00
...,...,...,...,...,...
1759,3.50,4.0,3,3.5,3.50
1760,3.50,4.0,3,3.5,3.50
1761,3.75,4.0,3,3.8,3.75
1762,3.75,4.0,3,3.8,3.75


In [ ]:
DECLARE @number DECIMAL(4, 2) = 4.5;
DECLARE @power INT = 4;

SELECT
@number AS number,
@power AS power,
-- Raise the @number to the @power
POWER(@number, @power) AS number_to_power,
-- Calculate the square of the @number
SQUARE(@number) num_squared,
-- Calculate the square root of the @number
SQRT(@number) num_square_root;

In [50]:
rsq("""-- SQLite syntax
-- Variable declaration is not supported in SQLite

SELECT 
       4.5 as number, 
       4 as power,
       -- Raise the number to the power
       CAST(POWER(4.5, 4) as decimal(4,2)) as number_to_power,
       -- Calculate the square of the number
       CAST(4.5 * 4.5 as decimal(4,2)) as num_squared,
       -- Calculate the square root of the number
       CAST(SQRT(4.5) as decimal(4,2)) as num_square_root;""")

,number,power,number_to_power,num_squared,num_square_root
0,4.5,4,410.0625,20.25,2.12132


In [ ]:
SELECT
company,
-- Select the number of cocoa flavors for each company
COUNT(*) AS flavors,
-- Select the minimum, maximum and average rating
MIN(rating) AS lowest_score,
MAX(rating) AS highest_score,
AVG(rating) AS avg_score,
-- Round the average rating to 1 decimal
ROUND(AVG(rating), 1) AS round_avg_score,
-- Round up and then down the aveg. rating to the next integer
CEILING(AVG(rating)) AS round_up_avg_score,
FLOOR(AVG(rating)) AS round_down_avg_score
FROM ratings
GROUP BY company
ORDER BY flavors DESC;

In [51]:
rsq("""SELECT
company,
-- Select the number of cocoa flavors for each company
COUNT(*) AS flavors,
-- Select the minimum, maximum and average rating
MIN(rating) AS lowest_score,
MAX(rating) AS highest_score,
AVG(rating) AS avg_score,
-- Round the average rating to 1 decimal
ROUND(AVG(rating), 1) AS round_avg_score,
-- Round up and then down the aveg. rating to the next integer
CEILING(AVG(rating)) AS round_up_avg_score,
FLOOR(AVG(rating)) AS round_down_avg_score
FROM ratings
GROUP BY company
ORDER BY flavors DESC;""")

,company,flavors,lowest_score,highest_score,avg_score,round_avg_score,round_up_avg_score,round_down_avg_score
0,Soma,47,2.00,4.00,3.015957,3.0,4.0,3.0
1,Bonnat,27,2.75,5.00,3.277778,3.3,4.0,3.0
2,Fresco,26,2.00,4.00,3.423077,3.4,4.0,3.0
3,Pralus,25,2.50,3.75,3.040000,3.0,4.0,3.0
4,A. Morin,23,2.50,4.00,3.250000,3.3,4.0,3.0
...,...,...,...,...,...,...,...,...
408,Baravelli's,1,3.50,3.50,3.500000,3.5,4.0,3.0
409,Artisan du Chocolat (Casa Luker),1,4.00,4.00,4.000000,4.0,4.0,4.0
410,Animas,1,3.25,3.25,3.250000,3.3,4.0,3.0
411,Anahata,1,2.75,2.75,2.750000,2.8,3.0,2.0
